In [ ]:
# 找桥

![](2023-05-17-18-28-29.png)

In [ ]:
class Solution:

    def criticalConnections(self, n: int, connections: List[List[int]]) -> List[List[int]]:
        adj = defaultdict(list)
        for x, y in connections:
            adj[x].append(y)
            adj[y].append(x)
        dfn = [0] * n
        low = [0] * n
        time = 0
        ans = []

        def tarjan(x, fa):
            nonlocal time
            time += 1
            dfn[x] = low[x] = time  # 入x 盖时间戳
            for y in adj[x]:
                if y == fa: continue  # 不回头访问
                if not dfn[y]:
                    tarjan(y, x)
                    # y回x 更新low 和 加桥(因为y已经处理好了)
                    low[x] = min(low[x], low[y])
                    if dfn[x] < low[y]:  # y没法回到x也就是桥 (x是环顶的父亲)
                        # 反向想 dfn[x] >= low[y] 说明 y可以到达x或x的祖先节点 所以x在环内
                        # 反过来 x 就在环外了
                        ans.append([x, y])
                else:  # 遇环 到x或x祖先 也更新
                    low[x] = min(low[x], dfn[y])

        tarjan(0, -1)
        return ans

In [ ]:
# 板子
class Solution:

    def criticalConnections(self, n: int, connections: List[List[int]]) -> List[List[int]]:

        def check_graph(edge, n):
            # edge: 边连接关系 [[],..] n:节点数

            # 访问序号与根节点序号
            visit = [0] * n
            root = [0] * n
            # 割点
            cut_node = []
            # 割边
            cut_edge = []
            # 强连通分量子树
            sub_group = []

            # 中间变量
            stack = []
            index = 1
            in_stack = [0] * n

            def tarjan(i, father):
                nonlocal index
                visit[i] = root[i] = index
                index += 1
                stack.append(i)

                in_stack[i] = 1
                child = 0
                for j in edge[i]:
                    if j != father:
                        if not visit[j]:
                            child += 1
                            tarjan(j, i)
                            x, y = root[i], root[j]
                            root[i] = x if x < y else y
                            # 割边 low[i] < dfn[i]
                            if visit[i] < root[j]:
                                cut_edge.append(sorted([i, j]))
                            # 两种情况下才为割点 low[i] <= dfn[i]
                            if father != -1 and visit[i] <= root[j]:
                                cut_node.append(i)
                            elif father == -1 and child >= 2:
                                cut_node.append(i)
                        elif in_stack[j]:
                            x, y = root[i], visit[j]
                            root[i] = x if x < y else y

                if root[i] == visit[i]:
                    lst = []
                    while stack[-1] != i:
                        lst.append(stack.pop())
                        in_stack[lst[-1]] = 0
                    lst.append(stack.pop())
                    in_stack[lst[-1]] = 0
                    r = min(root[ls] for ls in lst)
                    for ls in lst:
                        root[ls] = r
                    lst.sort()
                    sub_group.append(lst[:])
                return

            for k in range(n):
                if not visit[k]:
                    tarjan(k, -1)
            cut_edge.sort()
            cut_node.sort()
            sub_group.sort()
            return cut_edge, cut_node, sub_group

        edges = [[] for _ in range(n)]
        for a, b in connections:
            edges[a].append(b)
            edges[b].append(a)
        cut_edge, cut_node, sub_group = check_graph(edges, n)
        return cut_edge


# 作者：liupengsay
# 链接：https://leetcode.cn/problems/critical-connections-in-a-network/solutions/1767372/by-liupengsay-dlc2/
# 来源：力扣（LeetCode）
# 著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。